File where code is written, tested, and integrated. \
Middlebury Stereo Datasets: https://vision.middlebury.edu/stereo/data/

In [1]:
import cv2
import numpy as np
import sklearn 
import igraph

In [ ]:
class Configuration:
    def __init__(self, size, max_disparity):
        self.size = size
        self.max_disparity = max_disparity
        self.config_map = np.zeros((size[0]* size[1], max_disparity))


In [1]:
class Energy:
    def __init__(self):
        self.graph = igraph.Graph()
        self.graph.add_vertices(2) # source vertex:0, terminal vertex:1
        self.node_id = {}
        self.Econst = 0
        #igraph requires "preprocessing" to initialize edge with weights
        #see https://igraph.org/python/doc/tutorial/generation.html "From nodes to edges" for more detail
        self.vertex_num = 2
        self.edges = []
        self.capacity = []

    def get_node_id(self, x)->int: 
        return self.node_id[x]
    
    # Add a new binary variable as a node
    def add_node(self, x, E0, E1)->int:
        x_id = self.vertex_num
        self.node_id[x] = x_id
        self.vertex_num += 1
        self.addterm1(x, E0,E1)
        return x_id

    # Add a constant to the energy function
    def add_constant(self, E)->None:
        self.Econst += E
        return

    # Add a term E(x) of one binary variable to the energy function,
    # where E(0) = E0, E(1) = E1. E0 and E1 can be arbitrary.
    def add_term1(self, x, E0, E1)->None:
        x_id = self.node_id[x]
        self.edges.append([0,x_id], [x_id,1])
        self.capacity.append([E1], [E0])
        return
    
    # Add a  term E(x, y) of two binary variables to the energy function,
    # where E(0, 0) = A, E(0, 1) = B, E(1, 0) = C, E(1, 1) = D.
    # The term must be regular, i.e. E00 + E11 <= E01 + E10
    def add_term2(self, x, y, E00, E01, E10, E11)->None:
        # E = A B = B B + A-B 0 +    0    0
        #     C D   D D   A-B 0   B+C-A-D 0
        self.add_term1(x, E11, E01)
        self.add_term1(y, 0, E00 - E01)

        x_id = self.node_id[x]
        y_id = self.node_id[y]
        self.edges.append([x_id, y_id], [y_id, x_id])
        self.capacity.append([0], [E01 + E10 - E00 - E11])
        return

    # Forbid (x, y) = (0, 1) by putting infinite value to
    # the edge from x to y
    def forbid01(self, x, y)->None:
        x_id = self.node_id[x]
        y_id = self.node_id[y]
        self.edges.append([x_id, y_id], [y_id, x_id])
        self.capacity.append([np.inf], [0])
        return
    
    def minimize(self)->int:
        return 0

In [3]:
g = igraph.Graph()
g.add_vertices(3)
g.add_edges([(0,1), (1,2)])

In [4]:
print(g)

IGRAPH U--- 3 2 --
+ edges:
0--1 1--2
